In [3]:
from pylab import *
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import caffe
import os
import PIL
import scipy.io as sio
import tempfile
import numpy as np
from PIL import Image

In [2]:
#nyud_data_root = '/home/yangyong/fcn.berkeleyvision.org/data/nyud'

In [4]:
import numpy as np
from PIL import Image
import scipy.io

import random

mean_bgr = np.array((116.190, 97.203, 92.318), dtype=np.float32)
mean_hha = np.array((194.421, 90.766, 126.557), dtype=np.float32)
mean_d = np.array((106.768,), dtype=np.float32)

def load_sceneLabel(nyud_dir,idx):
        """
        Load scene classification label 
        """
        allSceneLabel = scipy.io.loadmat('{}/sceneLabel/imgAllScene.mat'.format(nyud_dir))
        scenelabel = allSceneLabel['gtScene'][0][int(idx)-5001]
        return np.array([scenelabel])
def load_image(nyud_dir, idx):
        """
        Load input image and preprocess for Caffe:
        - cast to float
        - switch channels RGB -> BGR
        - subtract mean
        - transpose to channel x height x width order
        """
        im = Image.open('{}/data/images/img_{}.png'.format(nyud_dir, idx))
        in_ = np.array(im, dtype=np.float32)
        in_ = in_[:,:,::-1]
        in_ -= mean_bgr
        in_ = in_.transpose((2,0,1))
        return in_
    
def load_depth(nyud_dir, idx):
        """
        Load pre-processed depth for NYUDv2 segmentation set.
        """
        #im = Image.open('{}/depth/img_{}.png'.format(self.nyud_dir, idx))
        im = scipy.io.loadmat('{}/data/depth/img_{}.mat'.format(nyud_dir, idx))
        d = im['depths'].astype(np.float32)
        #d = np.log(d)
        d -= mean_d
        d = d[np.newaxis, ...]
        return d
    
def load_hha(nyud_dir, idx):
        """
        Load HHA features from Gupta et al. ECCV14.
        See https://github.com/s-gupta/rcnn-depth/blob/master/rcnn/saveHHA.m
        """
        im = Image.open('{}/data/hha/img_{}.png'.format(nyud_dir, idx))
        hha = np.array(im, dtype=np.float32)
        hha -= mean_hha
        hha = hha.transpose((2,0,1))
        return hha
    
allSceneLabel = sio.loadmat('{}/sceneLabel/imgAllScene.mat'.format(nyud_data_root))

In [7]:
allSceneLabel = sio.loadmat('{}/sceneLabel/imgAllScene.mat'.format(nyud_data_root))
scene_label_names = allSceneLabel['sceneName']
scene_label_names = scene_label_names.flatten().flatten()
for i in range(len(scene_label_names)):
    scene_label_names[i] = scene_label_names[i][0]

In [44]:
all_labels = allSceneLabel['gtScene'][0]
all_labels = all_labels - 1 # Make labels start from 0

In [116]:
# static analyse
label_hist,_ = np.histogram(all_labels,bins = range(27))

In [47]:
label_hist

array([  7, 121, 383,  36,   5,  49,   6,   5,   4, 117,   3,   4,  27,
        50,   5,   2, 225,   3, 221,  78,  10,  31,   3,  17,   5,  25,   7])

In [118]:
label_hist.shape

(26,)

# Data clean, drop the classes that number is less than 10

1. new labels list
2. new scene name list
3. new index of picture

In [48]:
scenes_lists = []
for i in xrange(27):
    scenes_lists.append([])
for i in xrange(len(all_labels)):
    scenes_lists[int(all_labels[i])].append(i)

In [102]:
labels_new = []
scene_names_new = []
indexes_new = []
current_label = 0
i=0
while(i<27):
    if len(scenes_lists[i])>9:
        scene_names_new.append(scene_label_names[i])
        for j in range(len(scenes_lists[i])):
            labels_new.append(current_label)
            indexes_new.append(scenes_lists[i][j])
        current_label +=1
    i += 1 

In [67]:
len(indexes_new)

1390

In [112]:
nyud_dir = '/home/yangyong/fcn.berkeleyvision.org/data/nyud'

In [109]:
scene_names_new = np.array(scene_names_new)
labels_new = np.array(labels_new)
sio.savemat('new_data/label/scene_name.mat',{'scene_name':scene_names_new})
sio.savemat('new_data/label/label.mat',{'label':labels_new})

In [139]:
for i in range(len(indexes_new)):
    idx = indexes_new[i] + 5001
    #color = Image.open('{}/data/images/img_{}.png'.format(nyud_dir, idx))
    depth = sio.loadmat('{}/data/depth/img_{}.mat'.format(nyud_dir, idx))['depths']
    #hha  = Image.open('{}/data/hha/img_{}.png'.format(nyud_dir, idx))
    depth = Image.fromarray(depth)
    #color.save('new_data/data/color/img_{}.png'.format(i+5001))
    depth.save('new_data/data/depth/img_{}.png'.format(i+5001))
    #hha.save('new_data/data/hha/img_{}.png'.format(i+5001))
    